In [ ]:
import os
import geopandas as gpd
import pandas as pd
import scrapbook as sb
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from flowclient import connect, get_result_by_query_id, get_geography

%matplotlib inline

In [ ]:
# Default parameter values
api_url = "http://localhost:9090"
aggregation_unit = "admin2"
anchor_date = "2016-01-03"
previous_notebook = "inputs/notebooks/run_flows.ipynb"

In [ ]:
conn = connect(url=api_url, token=os.environ["FLOWCLIENT_TOKEN"])

In [ ]:
nb = sb.read_notebook(previous_notebook)
query_ids = nb.scraps["query_ids"].data
cache_results = {}
for key, val in query_ids.items():
    cache_results[key] = get_result_by_query_id(connection=conn, query_id=val)

In [ ]:
agg_unit_geodata = gpd.GeoDataFrame().from_features(get_geography(connection=conn, aggregation_unit=aggregation_unit))

In [ ]:
display(Markdown(f"# Day-to-day {aggregation_unit} flows - {anchor_date}"))

In [ ]:
flowabnormal = cache_results['flowabnormal'].set_index(["pcod_from", "pcod_to"])
flownormal = cache_results['flownormal'].set_index(["pcod_from", "pcod_to"])
flows_above_normal = (
    flowabnormal - flownormal
).dropna().reset_index()

inflow_above_normal = (
    flows_above_normal[
        flows_above_normal.pcod_from != flows_above_normal.pcod_to
    ]
    .groupby("pcod_to")
    .sum()
    .reset_index()
)
inflow_above_normal = gpd.GeoDataFrame(
    pd.merge(
        inflow_above_normal, agg_unit_geodata, left_on='pcod_to', right_on='pcod'
    )
)

outflow_above_normal = (
    flows_above_normal[
        flows_above_normal.pcod_from != flows_above_normal.pcod_to
    ]
    .groupby("pcod_from")
    .sum()
    .reset_index()
)
outflow_above_normal = gpd.GeoDataFrame(
    pd.merge(
        outflow_above_normal, agg_unit_geodata, left_on='pcod_from', right_on='pcod'
    )
)


## Inflows above normal

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(35, 7))
agg_unit_geodata.plot(ax=ax, color='grey')
inflows_plot = inflow_above_normal.plot(ax=ax, column='count', legend=True)
plt.axis("off")
fig.show()

In [ ]:
inflow_above_normal[['count', 'pcod']].style.hide_index()

## Outflows above normal

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(35,7))
agg_unit_geodata.plot(ax=ax, color='grey')
outflow_above_normal.plot(ax=ax, column='count', legend=True)
plt.axis("off")
fig.show()

In [ ]:
outflow_above_normal[['count', 'pcod']].style.hide_index()